# Postupci nelinearne optimizacije

In [1]:
import sys
sys.path.append('../')
from FirstLab.Matrica import Matrica, JedinicnaMatrica
from functools import reduce
import math
import numpy as np

In [2]:
# Ulazne velicine:
# - tocka: pocetna tocka pretrazivanja
# - h: pomak pretrazivanja
# - f: ciljna funkcija


def unimodalni(x0, func, h):
    
    tocka = x0[0]
    left = tocka - h 
    right = tocka + h
    m = tocka
    step = 1
    
    fm = func.vrijednost(tocka)
    fl = func.vrijednost(left)
    fr = func.vrijednost(right)
    
    if fm < fr and fm < fl: 
        return left, right
    elif fm > fr:
        while True:
            left = m
            m = right
            fm = fr
            step *= 2
            right = tocka + (h * step)
            fr = func.vrijednost(right)
            if (fm <= fr): 
                break
    else: 
        while True:
            right = m
            m = left
            fm = fl
            step *= 2
            left = tocka - (h * step)
            fl = func.vrijednost(left)
            if fm <= fl: break
    return left, right

In [3]:
# Ulazne velicine:
# - a, b: pocetne granice unimodalnog intervala
# - e: preciznost

def zlatni_rez(a, b, func, epsilon = 1e-6):
    
    k = 0.5 * (math.sqrt(5) - 1)
        
    c = b - k * (b - a)
    d = a + k * (b - a)
    fc = func.vrijednost(c)
    fd = func.vrijednost(d)
    
    while((b - a) > epsilon):
        if fc < fd:
            b = d 
            d = c
            c = b - k * (b - a)
            fd = fc
            fc = func.vrijednost(c)
        else:
            a = c
            c = d
            d = a + k * (b - a)
            fc = fd
            fd = f.vrijednost(d)
    return (a + b) / 2

In [4]:
def pret_po_koord_osima(x0, e, func):
    n = len(x0)
    x = Matrica(n, 1, x0)
    l = 0.0
    
    xs = x.copy()
    
    while True:
        xs = x.copy()
        for i in range(n):
            a, b = unimodalni(x0[0], func, 1.0)
            l = zlatni_rez(a, b, func)
            x[i][0] += l
        
        

def pretraga(elementi):
    l = elementi[0]
    x[i][0] += l
    rezultat = func.vrijednost(x)
    x[i][0] -= l
    return rezultat
    


In [5]:
def simplex(x0, func, pomak = 1, alfa = 1.0, beta = 0.5, gama = 2.0, sigma = 0.5, epsilon=1e-6):
    
    #tocke simplexa

    X = [Matrica(len(x0), 1, x0)]
    for i in len(x0):
        nova_tocka = Matrica(len(x0), 1, x0)
        nova_tocka[i] += pomak
        X.append(nova_tocka)
    
    # - h, l, centroid -
    while True:
        h = index_min(X, func)
        l = index_max(X, func)
        xc = centroid(X, h, f)
        xr = refleksija(xc, X[h], alfa)
        
        if func.vrijednost(xr) < func.vrijednost(X[l]):
            xe = ekspanzija(xr, xc, gama)
            
            if func.vrijednost(xe) < func.vrijednost(X[l]):
                X[h] = xe
            else:
                X[h] = xr

        else:
            # provjera
            flag = True
            for j in range(len(X)): 
                if j != h:
                    if f_xr <= func.vrijednost(X[j]):
                        flag = False
                        break
            if flag:
                if func.vrijednost(xr) < func.vrijednost(X[h]):
                    X[h] = xr

                xk = kontrakcija(X[h], xc, beta)

                if func.vrijednost(xk) < func.vrijednost(X[h]):
                    X[h] = xk
                else:
                    pomak_prema_xl(X, l, sigma)
            else:
                X[h] = xr
                
        # Dodati uvjet zaustavljanja
        break        
            
        
# - MINIMUM I MAKSIMUM FJE CILJA        
def index_min(X, f):
    i_min = 0
    for i in range(len(X)):
        if func.vrijednost(X[i]) < func.vrijednost(X[i_min]): i_min = i
    return i_min
 
def index_max(X, f):
    i_min = 0
    for i in range(len(X)):
        if func.vrijednost(X[i]) > func.vrijednost(X[i_min]): i_min = i
    return i_min

# - CENTROID -
def centroid(X, h, f):
    tocke = X.copy()
    del tocke[h]
    return (1.0 / len(tocke)) * reduce(myadd, tocke)

def myadd(a, b):
    return a.elementi + b.elementi

# - REFLEKSIJA, EKSPANZIJA, KONTRAKCIJA -
def refleksija(xc, xh, alfa = 1.0):
    return ((1.0 + alfa) * xc) - (alfa * xh)

def ekspanzija(xc, xr, gama = 2.0):
    return ((1.0 - gama) * xc) + (gama * xr)

def kontrakcija(xh, xc, beta = 0.5):
    return ((1.0 - beta) * xc) + (beta * xh)

def pomak_prema_xl(X, l, sigma = 0.5):
    for i in range(len(X)):
        X[i] = 0.5 * (X[i] + X[l])
    return X

In [6]:
sim = [Matrica(1, 2, [2, 3]), Matrica(1, 2, [1, 1]), Matrica(1, 2, [3, 4]), Matrica(1, 2, [4, 5]), Matrica(1, 2, [5, 6])]
novi = pomak_prema_xl(sim, 1)
for m in novi:
    print(m)
    print(m[0])

[(2, 1), (3, 1)]
[3, 4]
[(1, 1), (1, 1)]
[2, 2]
[(3, array([1., 1.]))]
[array([4., 4.])]
[(4, array([1., 1.]))]
[array([5., 5.])]
[(5, array([1., 1.]))]
[array([6., 6.])]
[[1.5 2. ]]
[1.5 2. ]
[[1. 1.]]
[1. 1.]
[[2. 2.]]
[2. 2.]
[[2.5 2.5]]
[2.5 2.5]
[[3. 3.]]
[3. 3.]


In [29]:
# x0 - početna točka
# xb - bazna točka
# xp - početna točka pretraživanja
# xn - točka dobivena pretraživanjem

def hooke_jeeves(x0, f, dx = 0.5, e = 1e-6):
    
    if (type(x0) == int or type(x0) == float):
        xp = x0
        xb = x0
        dx = 0.5
    else:
        xp = x0.copy()
        xb = x0.copy()
        dx = np.full((len(x0), 1), 0.5)
        
    while True:
        xn = istrazi(xp, dx)
        if func.vrijednost(xn) < func.vrijednost(xb):
            xp = 2*xn - xb
            xb = xn.copy()
        else:
            #smanji dx?
            xp = xb.copy()
    
        #uvjet zaustavljanja
    rjesenje = xb.copy()

def istrazi(xp, dx):
    x = xp.copy()
    for i in range(len(x)):
        p = func.vrijednost(x)
        x[i] += dx
        n = func.vrijednost(x)
        if n > p:
            x[i] -= 2 * dx
            n = func.vrijednost(x)
            if n > p:
                x[i] += dx
    return x

In [2]:
class Funkcija:
    
    def __init__(self, f):
        self.br_poziva = 0
        self.f = f
        self.vrijednosti = {}
    
    def brojac(self):
        return self.br_poziva
    
    def reset(self):
        self.br_poziva = 0
    
    def vrijednost(self, x):
        vrijednost = self.func.vrijednost(x)
        if x not in self.vrijednosti:
            self.vrijednosti[x] = vrijednost
        self.br_poziva += 1
        return vrijednost

def f1(x):
    return 100 * (x[1] - (x[0])**2)**2 + (1 - x[0])**2

def f2(x):
    return (x[0] - 4)**2 + 4 * (x[1] - 2)**2

def f3(x):
    result = 0
    try;
        for i in len(x):
            result += (x[i] - i + 1)**2
    except TypeError:
        result = (x - 1)**2
    return result

def f4(x):
    return abs((x[0] - x[1]) * (x[0] + x[1])) + math.sqrt(x[0]**2 + x[1]**2)

def f6(x):
    suma = 0
    try:
        for i in range(len(x)):
            suma += x[i]**2
    except: 
        suma = x**2
    sinus = math.sin(math.sqrt(suma))
    razlomak = ((sinus**2) - 0.5) / ((1 + 0.001 * suma)**2)
    return 0.5 + razlomak

### Provjere

In [ ]:
# Provjera mnozenja matrica

v1 = Matrica(1, 3, [1, 2, 3])
v2 = Matrica(3, 1, [1, 2, 3])
m1 = Matrica(3, 3, [[1, 2, 3], [1, 1, 1], [1, 2, 3]])

print("m1/3")
print(m1/3)

print("1/3 * (m1)")
print(1/3 * m1)

# print(m1*v1)
print("m1*v2")
print(m1*v2)
print("v1*m1")
print(v1*m1)
print("m1*m1")
print(m1*m1)
print("v1*v2")
print(v1*v2)
print("v2*v1")
print(v2*v1)